In [5]:
import glob
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from glob import glob
from spreadsurfer.price_engine import FeatureEngineer
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler

## chosen models
# min: models/2023-02-03T22:09:25-min_quantile0.5-score-73.0.cat
# max: models/2023-02-03T22:14:13-max_quantile0.7-score--154.0.cat


# cat_filename = 'models/2023-02-03T22:09:25-max_quantile0.5-score-77.0.cat'
# cat_filename = 'models/2023-02-03T22:14:13-max_quantile0.7-score--154.0.cat'

# cat_filename = 'models/2023-02-03T18:06:57-max_mape-score--21.0.cat'

# cat_filename = 'models/2023-02-03T21:57:57-min_huber0.4-score-108.0.cat'
# cat_filename = 'models/2023-02-03T21:34:07-min_poisson-score-191.0.cat'
# cat_filename = 'models/2023-02-03T21:50:04-min_quantile0.3-score--135.0.cat'
# cat_filename = '2023-02-03T20:13:00-min_rmse-score--10.0.cat'
# cat_filename = 'models/2023-02-01T09:14:50-mape-score-161.0.cat'

def load_and_score_mae(cat_filename, min=False):
    model = CatBoostRegressor()
    model.load_model(fname=cat_filename)
    print(f'catboost model loaded from {cat_filename}')

    pipeline = Pipeline(steps=[
        ('preprocessor', FeatureEngineer()),
        ('model', model)
    ])

    with open("neverseen.log","r") as f:
        input_from_log = ''.join([x for x in f.readlines() if 'collected' in x][-100:])

    mae = 0
    count = 0
    for line in input_from_log.split('\n'):
        if not line: continue
        count += 1
        json = '{' + line.split('{')[1]
        json = json.replace('nan', '0')
        sample = pd.DataFrame([eval(json)])
        if sample.wave_direction[0] not in ['min', 'max']: continue

        real = sample.last_price_delta_since_stabilized[0]
        if abs(real) < 0.2: continue
        sample.drop('last_price_delta_since_stabilized', axis=1, inplace=True)

        sample.loc[sample['wave_direction'] == 'min', 'wave_direction'] = 1
        sample.loc[sample['wave_direction'] == 'max', 'wave_direction'] = -1
        sample = sample.astype({"wave_direction": 'float64'})

        # if sample['wave_direction'][0] != (1 if min else -1):
        #     continue

        print('expected: ', real)
        guess = model.predict(sample)[0]
        guess *= 3.5
        print('predict: ', guess)
        mae += abs(real - guess)
        print()
    display(f'MAE: {mae / count}')
    return mae / count

# load_and_score_mae(cat_filename)

## search for best model
# models = [x for x in glob('models/*max_quantile*.cat')]
models = [x for x in glob('models/*')]
display(models)
results = {}
for i in models:
    try:
        mae = load_and_score_mae(i, min=True)
        results[mae] = i
    except Exception:
        print('skip with error')

scored = dict(sorted(results.items()))
print('\n'.join(scored.values()))


['models/2023-01-21T20:37:02-score-773.0.cat',
 'models/2023-01-24T07:03:25-score-530.0.cat',
 'models/2023-01-24T11:19:53-score-449.0.cat',
 'models/readme.md',
 'models/2023-01-24T15:37:36-score-384.0.cat',
 'models/2023-01-24T15:38:18-score-462.0.cat',
 'models/2023-01-31T23:21:57-mape-score-174.0.cat',
 'models/2023-01-31T23:21:57-rmse-score-314.0.cat',
 'models/2023-02-01T09:14:50-mape-score-161.0.cat',
 'models/2023-02-01T09:14:50-rmse-score-291.0.cat',
 'models/2023-02-03T18:06:57-min_mape-score--17.0.cat',
 'models/2023-02-03T18:06:57-max_mape-score--21.0.cat',
 'models/2023-02-03T20:13:00-min_rmse-score--10.0.cat',
 'models/2023-02-03T20:13:00-max_rmse-score--23.0.cat',
 'models/2023-02-03T21:34:07-min_poisson-score-191.0.cat',
 'models/2023-02-03T21:34:07-max_poisson-score--324.0.cat',
 'models/2023-02-03T21:48:03-min_quantile0.9-score--674.0.cat',
 'models/2023-02-03T21:48:03-max_quantile0.9-score--312.0.cat',
 'models/2023-02-03T21:50:04-min_quantile0.3-score--135.0.cat',
 

catboost model loaded from models/2023-01-21T20:37:02-score-773.0.cat
expected:  -0.25
predict:  4.202502559424064

expected:  0.64
predict:  7.613658881625833

expected:  0.55
predict:  6.552490654248354

expected:  -0.5
predict:  3.1016731074123287

expected:  -0.23
predict:  3.9442641232292615

expected:  -1.59
predict:  4.406362388612905

expected:  1.14
predict:  8.268997228000348

expected:  0.94
predict:  7.483199953994003

expected:  -0.54
predict:  5.617409732239828

expected:  -0.21
predict:  4.129458710270769

expected:  -0.33
predict:  4.637071415499197

expected:  0.21
predict:  5.4016821680988185

expected:  -0.43
predict:  4.598578671678233

expected:  -1.01
predict:  4.479604018285443

expected:  -0.2
predict:  5.2659643142762915

expected:  3.18
predict:  8.584923326026722

expected:  -0.3
predict:  4.325444671708212

expected:  0.92
predict:  8.396993034969423

expected:  0.27
predict:  7.159202152036877

expected:  1.73
predict:  9.011077348349799

expected:  -0.39
p

'MAE: 1.4889106314130984'

catboost model loaded from models/2023-01-24T07:03:25-score-530.0.cat
expected:  -0.25
predict:  0.382352773824196

expected:  0.64
predict:  1.8272762172578914

expected:  0.55
predict:  1.547867074826005

expected:  -0.5
predict:  1.1691395554974673

expected:  -0.23
predict:  1.054978897378123

expected:  -1.59
predict:  0.4072578130596336

expected:  1.14
predict:  5.172253485565069

expected:  0.94
predict:  6.348277749352251

expected:  -0.54
predict:  -0.11943813931212538

expected:  -0.21
predict:  0.5194924700750753

expected:  -0.33
predict:  0.11292895416272607

expected:  0.21
predict:  5.835045506377005

expected:  -0.43
predict:  -0.13686171257765983

expected:  -1.01
predict:  0.35195211664946596

expected:  -0.2
predict:  0.7065592958590663

expected:  3.18
predict:  12.503586570521973

expected:  -0.3
predict:  0.0579183417860884

expected:  0.92
predict:  2.620510412779525

expected:  0.27
predict:  0.7459070421009879

expected:  1.73
predict:  7.3814536852041215

exp

'MAE: 0.5189223011079135'

...

catboost model loaded from models/2023-02-08T22:17:55-max_mape-score--27.0.cat
expected:  -0.25
predict:  -0.6885300947419789

expected:  0.64
predict:  -1.128362522027909

expected:  0.55
predict:  -1.5625265945625428

expected:  -0.5
predict:  -0.541135378836562

expected:  -0.23
predict:  -0.5322235991686751

expected:  -1.59
predict:  -0.9695686646431212

expected:  1.14
predict:  -1.1311156829579123

expected:  0.94
predict:  -0.6766161004963325

expected:  -0.54
predict:  -0.34337598979731376

expected:  -0.21
predict:  -2.0692855907763343

expected:  -0.33
predict:  -1.0274313851888834

expected:  0.21
predict:  -1.7658126972044264

expected:  -0.43
predict:  -1.2943902758271013

expected:  -1.01
predict:  -0.5473832090539393

expected:  -0.2
predict:  -2.057032330122926

expected:  3.18
predict:  -2.991466026580285

expected:  -0.3
predict:  -0.4446921794576725

expected:  0.92
predict:  -0.9384265872742588

expected:  0.27
predict:  -0.971504105782308

expected:  1.73
predict:

'MAE: 0.34384000215959193'

catboost model loaded from models/2023-02-09T20:05:57-mape-score-126.0.cat
expected:  -0.25
predict:  -0.33381900802663705

expected:  0.64
predict:  0.3336713274390162

expected:  0.55
predict:  0.4827536632059254

expected:  -0.5
predict:  -0.31529032645272353

expected:  -0.23
predict:  -0.5161163142562414

expected:  -1.59
predict:  -0.3551902269828745

expected:  1.14
predict:  1.1380419517450282

expected:  0.94
predict:  0.8185788298481572

expected:  -0.54
predict:  -0.6295173418783049

expected:  -0.21
predict:  -0.38243273635497005

expected:  -0.33
predict:  -0.21370283745824653

expected:  0.21
predict:  1.4007142361503633

expected:  -0.43
predict:  -0.48145900839721256

expected:  -1.01
predict:  -0.43318889911389374

expected:  -0.2
predict:  -0.5021814621222135

expected:  3.18
predict:  0.1477227994364838

expected:  -0.3
predict:  -0.22451048839569404

expected:  0.92
predict:  0.009160082248520402

expected:  0.27
predict:  0.017790864704376037

expected:  1.73
predi

'MAE: 0.15387618899734767'

models/2023-02-09T20:05:57-mape-score-126.0.cat
models/2023-02-03T21:34:07-max_poisson-score--324.0.cat
models/2023-02-03T21:48:03-max_quantile0.9-score--312.0.cat
models/2023-02-01T09:14:50-rmse-score-291.0.cat
models/2023-01-31T23:21:57-mape-score-174.0.cat
models/2023-01-31T23:21:57-rmse-score-314.0.cat
models/2023-02-03T21:50:04-min_quantile0.3-score--135.0.cat
models/2023-02-03T22:14:13-max_quantile0.7-score--154.0.cat
models/2023-02-07T13:50:42-max_quantile0.7-score--173.0.cat
models/2023-02-01T09:14:50-mape-score-161.0.cat
models/2023-02-08T22:17:55-max_mape-score--27.0.cat
models/2023-02-03T18:06:57-min_mape-score--17.0.cat
models/2023-02-03T18:06:57-max_mape-score--21.0.cat
models/2023-02-03T22:09:25-min_quantile0.5-score-73.0.cat
models/2023-02-03T22:09:25-max_quantile0.5-score-77.0.cat
models/2023-02-08T22:17:55-min_-mape-score--8.0.cat
models/2023-02-03T21:57:57-max_huber0.4-score-105.0.cat
models/2023-02-03T21:57:57-min_huber0.4-score-108.0.cat
models/2023-01-24T07:03:25-s